In [ ]:
import plotly.express as px
import pandas as pd
from datatypes import *
from utils import *


# Example DataFrame
df = read_obc(os.path.join(os.getcwd(), "..", "ros2_ws", "src", "simulation", "worlds", "autogen.obc"))

# Create interactive scatter plot
fig = px.scatter_3d(
    df,
    x="x",
    y="y",
    z="z",
    hover_name="id",  # Show label on hover
    title="autogen.obc",
)
fig.update_layout(scene=dict(aspectmode="data"))  # Ensures x, y, z use the same scale
fig.update_traces(marker=dict(size=2))
fig.update_layout(width=800, height=600)  # default ~700  # increase vertical size
fig.show()

In [1]:
import os
from datatypes import *
import plotly.graph_objects as go
import json
from core import c4l5


def plot_camera(name="TopViewCameraFront"):
    vehicle, _ = c4l5()
    photogrammetry = read_obc("/" + os.path.join("home", "workspace", "datasets", "C4L5", "photogrammetry.obc"))

    name = "TopViewCameraFront"
    sensor_folder = "/" + os.path.join("home", "workspace", "datasets", "C4L5", name.lower())
    features = pd.DataFrame.from_dict(json.load(open(os.path.join(sensor_folder, "detections.json"))))
    features["x"] += 0.5
    features["y"] += 0.5
    camera = Camera(
        name=name,
        data=Image.open(os.path.join(sensor_folder, name.lower() + ".bmp")),
        intrinsics=Camera.Intrinsics.from_json(os.path.join(sensor_folder, "camera_info.json")),
        features=features,  # Groundtruth detections from simulation
    )

    fig = go.Figure()
    fig.add_trace(go.Image(z=np.asarray(camera.data)))
    fig.add_trace(go.Scatter(x=camera.features["x"],
                           y=camera.features["y"],
                           name="feature",
                           hovertext=camera.features["id"],
                           mode="markers",
                           marker=dict(size=15, symbol="x", color="green")))
    convention = SE3(rotation=Rotation.from_euler("YXZ", [90, 0, -90], degrees=True))
    camera_parameter = g2o.CameraParameters(camera.intrinsics.width / (2 * np.tan(np.deg2rad(140) / 2.0)), camera.intrinsics.principal_point, 0)
    mapped = []
    for guess in vehicle.children:
        for frame in guess.children:
            if type(frame) is Camera:
                if "front" in frame.name.lower():
                    overall_transform = guess.transform @ frame.transform

    ids = []
    for index, row in photogrammetry.iterrows():
        trow = (overall_transform @ convention).inverse().apply(row[["x", "y", "z"]].to_numpy())
        projection = camera_parameter.cam_map(trow)
        if projection[0] > 0.0 and projection[0] < camera.intrinsics.width and projection[1] > 0.0 and projection[1] < camera.intrinsics.height and trow[2] > 0.0:
            # mapped.append(np.round(projection * 2) / 2)
            mapped.append(projection)
            ids.append(row["id"])
    df_mapped = pd.DataFrame(mapped, columns=["x", "y"])
    df_mapped["id"] = ids
    fig.add_trace(
        go.Scatter(
            x=df_mapped["x"],
            y=df_mapped["y"],
            name="projections",
            hovertext=df_mapped["id"],
            mode="markers",
            marker=dict(size=8, symbol="x", color="red"),
        )
    )
    fig.update_layout(height=1000, width=1000)
    return fig

plot_camera()

iteration= 0	 chi2= 16.974184	 time= 0.00012885	 cumTime= 0.00012885	 edges= 204	 schur= 0	 lambda= 29.214326	 levenbergIter= 1
iteration= 1	 chi2= 16.974183	 time= 6.9416e-05	 cumTime= 0.000198266	 edges= 204	 schur= 0	 lambda= 19.476218	 levenbergIter= 1
iteration= 2	 chi2= 16.974183	 time= 6.6057e-05	 cumTime= 0.000264323	 edges= 204	 schur= 0	 lambda= 12.984145	 levenbergIter= 1
iteration= 3	 chi2= 16.974183	 time= 9.1693e-05	 cumTime= 0.000356016	 edges= 204	 schur= 0	 lambda= 17.312193	 levenbergIter= 2
iteration= 4	 chi2= 16.974183	 time= 0.000133327	 cumTime= 0.000489343	 edges= 204	 schur= 0	 lambda= 11818.457381	 levenbergIter= 5
iteration= 5	 chi2= 16.974183	 time= 0.000112126	 cumTime= 0.000601469	 edges= 204	 schur= 0	 lambda= 63031.772698	 levenbergIter= 3
iteration= 6	 chi2= 16.974183	 time= 8.8071e-05	 cumTime= 0.00068954	 edges= 204	 schur= 0	 lambda= 84042.363597	 levenbergIter= 2
iteration= 7	 chi2= 16.974183	 time= 0.000127072	 cumTime= 0.000816612	 edges= 204	 schu

          x             y         z          roll     pitch           yaw  \
0  0.000155  1.026956e-15  0.000345 -1.272222e-14  0.004569 -2.824900e-30   
1  0.000204 -1.013154e-16  0.000594 -1.590085e-14  0.023093  9.541024e-15   
2  0.000204  5.069358e-17  0.000594 -9.543587e-15  0.023093 -9.541024e-15   
3  0.000135 -4.992897e-04  0.000363 -4.164740e-03  0.005363  4.247211e-03   

                 name  depth                                     path  
0  TopViewCameraFront      2  Vehicle/InitialGuess/TopViewCameraFront  
1   TopViewCameraLeft      2   Vehicle/InitialGuess/TopViewCameraLeft  
2  TopViewCameraRight      2  Vehicle/InitialGuess/TopViewCameraRight  
3   TopViewCameraRear      2   Vehicle/InitialGuess/TopViewCameraRear  
